In [ ]:
from src.deep_research_agent.agents.research_agent import researcher_agent

/Users/gaurav/Documents/projects/OpenDeepResearch/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
